# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 10 2022 2:16PM,239356,10,Enova-10116,Emerginnova,Released
1,Jun 10 2022 2:11PM,239355,20,SSNA-2173662705,Sartorius Stedim North America,Released
2,Jun 10 2022 2:06PM,239354,10,8126336,Beach Products Inc,Released
3,Jun 10 2022 2:04PM,239353,20,ALUR038152223,Alumier Labs Inc.,Released
4,Jun 10 2022 2:04PM,239353,20,ALUR074904655,Alumier Labs Inc.,Released
5,Jun 10 2022 2:04PM,239353,20,ALUR119861207,Alumier Labs Inc.,Released
6,Jun 10 2022 2:04PM,239353,20,ALUR249088369,Alumier Labs Inc.,Released
7,Jun 10 2022 2:04PM,239353,20,ALUR379135987,Alumier Labs Inc.,Released
8,Jun 10 2022 2:04PM,239353,20,ALUR574029692,Alumier Labs Inc.,Released
9,Jun 10 2022 2:04PM,239353,20,ALUR822109220,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
24,239353,Executing,2
25,239353,Released,14
26,239354,Released,1
27,239355,Released,1
28,239356,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239351,NaN,23.0,13.0
239353,NaN,2.0,14.0
239354,NaN,NaN,1.0
239355,NaN,NaN,1.0
239356,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239294,0.0,0.0,1.0
239306,0.0,11.0,0.0
239313,0.0,15.0,5.0
239315,22.0,6.0,1.0
239318,0.0,0.0,11.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239294,0,0,1
239306,0,11,0
239313,0,15,5
239315,22,6,1
239318,0,0,11


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239294,0,0,1
1,239306,0,11,0
2,239313,0,15,5
3,239315,22,6,1
4,239318,0,0,11


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239294,,,1
1,239306,,11,
2,239313,,15,5
3,239315,22,6,1
4,239318,,,11


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 10 2022 2:16PM,239356,10,Emerginnova
1,Jun 10 2022 2:11PM,239355,20,Sartorius Stedim North America
2,Jun 10 2022 2:06PM,239354,10,Beach Products Inc
3,Jun 10 2022 2:04PM,239353,20,Alumier Labs Inc.
19,Jun 10 2022 1:58PM,239351,10,ISDIN Corporation
55,Jun 10 2022 1:55PM,239350,10,"Citieffe, Inc."
59,Jun 10 2022 1:41PM,239347,10,"Methapharm, Inc."
60,Jun 10 2022 1:39PM,239346,12,Hush Hush
64,Jun 10 2022 1:24PM,239342,10,"Senseonics, Incorporated"
65,Jun 10 2022 1:13PM,239340,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 10 2022 2:16PM,239356,10,Emerginnova,,,1
1,Jun 10 2022 2:11PM,239355,20,Sartorius Stedim North America,,,1
2,Jun 10 2022 2:06PM,239354,10,Beach Products Inc,,,1
3,Jun 10 2022 2:04PM,239353,20,Alumier Labs Inc.,,2,14
4,Jun 10 2022 1:58PM,239351,10,ISDIN Corporation,,23,13
5,Jun 10 2022 1:55PM,239350,10,"Citieffe, Inc.",,2,2
6,Jun 10 2022 1:41PM,239347,10,"Methapharm, Inc.",,,1
7,Jun 10 2022 1:39PM,239346,12,Hush Hush,,,4
8,Jun 10 2022 1:24PM,239342,10,"Senseonics, Incorporated",,,1
9,Jun 10 2022 1:13PM,239340,10,"Citieffe, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 10 2022 2:16PM,239356,10,Emerginnova,1,,
1,Jun 10 2022 2:11PM,239355,20,Sartorius Stedim North America,1,,
2,Jun 10 2022 2:06PM,239354,10,Beach Products Inc,1,,
3,Jun 10 2022 2:04PM,239353,20,Alumier Labs Inc.,14,2,
4,Jun 10 2022 1:58PM,239351,10,ISDIN Corporation,13,23,
5,Jun 10 2022 1:55PM,239350,10,"Citieffe, Inc.",2,2,
6,Jun 10 2022 1:41PM,239347,10,"Methapharm, Inc.",1,,
7,Jun 10 2022 1:39PM,239346,12,Hush Hush,4,,
8,Jun 10 2022 1:24PM,239342,10,"Senseonics, Incorporated",1,,
9,Jun 10 2022 1:13PM,239340,10,"Citieffe, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 10 2022 2:16PM,239356,10,Emerginnova,1,,
1,Jun 10 2022 2:11PM,239355,20,Sartorius Stedim North America,1,,
2,Jun 10 2022 2:06PM,239354,10,Beach Products Inc,1,,
3,Jun 10 2022 2:04PM,239353,20,Alumier Labs Inc.,14,2,
4,Jun 10 2022 1:58PM,239351,10,ISDIN Corporation,13,23,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 10 2022 2:16PM,239356,10,Emerginnova,1.0,NaN,NaN
1,Jun 10 2022 2:11PM,239355,20,Sartorius Stedim North America,1.0,NaN,NaN
2,Jun 10 2022 2:06PM,239354,10,Beach Products Inc,1.0,NaN,NaN
3,Jun 10 2022 2:04PM,239353,20,Alumier Labs Inc.,14.0,2.0,NaN
4,Jun 10 2022 1:58PM,239351,10,ISDIN Corporation,13.0,23.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 10 2022 2:16PM,239356,10,Emerginnova,1.0,0.0,0.0
1,Jun 10 2022 2:11PM,239355,20,Sartorius Stedim North America,1.0,0.0,0.0
2,Jun 10 2022 2:06PM,239354,10,Beach Products Inc,1.0,0.0,0.0
3,Jun 10 2022 2:04PM,239353,20,Alumier Labs Inc.,14.0,2.0,0.0
4,Jun 10 2022 1:58PM,239351,10,ISDIN Corporation,13.0,23.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2632754,56.0,31.0,22.0
12,478652,4.0,11.0,0.0
16,478637,20.0,0.0,0.0
20,957344,21.0,24.0,1.0
21,478614,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2632754,56.0,31.0,22.0
1,12,478652,4.0,11.0,0.0
2,16,478637,20.0,0.0,0.0
3,20,957344,21.0,24.0,1.0
4,21,478614,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,56.0,31.0,22.0
1,12,4.0,11.0,0.0
2,16,20.0,0.0,0.0
3,20,21.0,24.0,1.0
4,21,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,56.0
1,12,Released,4.0
2,16,Released,20.0
3,20,Released,21.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,20,21
Status,,,,,
Completed,22.0,0.0,0.0,1.0,0.0
Executing,31.0,11.0,0.0,24.0,0.0
Released,56.0,4.0,20.0,21.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,20,21
0,Completed,22.0,0.0,0.0,1.0,0.0
1,Executing,31.0,11.0,0.0,24.0,0.0
2,Released,56.0,4.0,20.0,21.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,20,21
0,Completed,22.0,0.0,0.0,1.0,0.0
1,Executing,31.0,11.0,0.0,24.0,0.0
2,Released,56.0,4.0,20.0,21.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()